In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import common
from search_tools import create_search_tools

evidently_search_tools = create_search_tools()

In [3]:
import inspect

search_tool = {
    "type": "function",
    "name": evidently_search_tools.search.__name__,
    "description": inspect.getdoc(evidently_search_tools.search),
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query parameter"
            }
        },
        "required": [
            "query"
        ]
    }
}

get_file_tool = {
    "type": "function",
    "name": evidently_search_tools.get_file.__name__,
    "description": inspect.getdoc(evidently_search_tools.get_file),
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "filename parameter"
            }
        },
        "required": [
            "filename"
        ]
    }
}

In [4]:
tool_pairs = [
    (evidently_search_tools.search, search_tool),
    (evidently_search_tools.get_file, get_file_tool)
]

In [5]:
tools = [t for (_, t) in tool_pairs]
tools_index = {t["name"]: f for (f, t) in tool_pairs}

In [6]:
def make_call(tool_call):
    name = tool_call.name

    if name in tools_index:
        func = tools_index[name]
        arguments = json.loads(tool_call.arguments)
        result = func(**arguments)
    else:
        result = f"Tool '{name}' not found in tools index"

    return {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": json.dumps(result)
    }

In [7]:
from dataclasses import dataclass

@dataclass
class ToolCall:
    name: str
    arguments: str
    call_id: str

tool_call = ToolCall(name="search", arguments='{"query": "dashboard"}', call_id="1234")
tool_call_result = make_call(tool_call)
tool_call_result

{'type': 'function_call_output',
 'call_id': '1234',
 'output': '[{"title": "Add dashboard panels (UI)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["**Dashboards** let you create Panels to...", "...ject.\\n\\n<Check>\\n  No-code **Dashboards** are available in the Evi...", "...anels appear on a single **Dashboard**. You can add multiple Ta..."], "total_matches": 11}, "filename": "docs/platform/dashboard_add_panels_ui.mdx"}, {"title": "Add dashboard panels (API)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["...oard_add_panels_ui).\\n\\n## **Dashboard** Management\\n\\n<Check>\\n  **Dashboards**...", "...rd Management\\n\\n<Check>\\n  **Dashboards** as code are available in...", "... Tab:\\n\\n```python\\nproject.**dashboard**.add_tab(\\"Another Tab\\")\\n`..."], "total_matches": 27}, "filename": "docs/platform/dashboard_add_panels.mdx"}, {"title": "Dashboard panel types [Legacy]", "descriptio

In [8]:
from openai import OpenAI
openai_client = OpenAI()

In [9]:
messages = [
    {"role": "system", "content": common.SINGLE_TOOL_SYSTEM_PROMPT},
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

# 1st call - initial

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    tools=tools,
)

In [10]:
response.output

[ResponseFunctionToolCall(arguments='{"query":"create a dashboard"}', call_id='call_2jzoK4oG4rGwPqIOh3eHzFsr', name='search', type='function_call', id='fc_0487122c6a3abcdd006990e794a678819d94ae1a9ab6dc8f9b', status='completed')]

In [11]:
tool_call = [m for m in response.output if m.type == 'function_call'][0]
tool_call

ResponseFunctionToolCall(arguments='{"query":"create a dashboard"}', call_id='call_2jzoK4oG4rGwPqIOh3eHzFsr', name='search', type='function_call', id='fc_0487122c6a3abcdd006990e794a678819d94ae1a9ab6dc8f9b', status='completed')

In [12]:
print(f"Tool call: {tool_call.name}({tool_call.arguments})")

Tool call: search({"query":"create a dashboard"})


In [13]:
tool_call_result = make_call(tool_call)
tool_call_result

{'type': 'function_call_output',
 'call_id': 'call_2jzoK4oG4rGwPqIOh3eHzFsr',
 'output': '[{"title": "Add dashboard panels (UI)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["**Dashboards** let you **create** Panels to...", "**Dashboards** let you **create** Panels to visualize eval...", "...ject.\\n\\n<Check>\\n  No-code **Dashboards** are available in the Evi..."], "total_matches": 14}, "filename": "docs/platform/dashboard_add_panels_ui.mdx"}, {"title": "Add dashboard panels (API)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["...oard_add_panels_ui).\\n\\n## **Dashboard** Management\\n\\n<Check>\\n  **Dashboards**...", "...rd Management\\n\\n<Check>\\n  **Dashboards** as code are available in...", "...(/docs/setup/cloud) and [**create** a Project](/docs/platfor..."], "total_matches": 32}, "filename": "docs/platform/dashboard_add_panels.mdx"}, {"title": "LLM as a judge", "description": "How t

In [14]:
messages.extend(response.output)
messages.append(tool_call_result)

In [16]:
# 2nd call

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    tools=tools,
)

In [17]:
response.output

[ResponseFunctionToolCall(arguments='{"filename":"docs/platform/dashboard_add_panels_ui.mdx"}', call_id='call_gp5tuxxHojj4AJYBa4CQndMg', name='get_file', type='function_call', id='fc_0487122c6a3abcdd006990e7d0c048819d9610d243bed954c0', status='completed'),
 ResponseFunctionToolCall(arguments='{"filename":"docs/platform/dashboard_add_panels.mdx"}', call_id='call_eikBzZZGv0Dh5XsyjxOUbd5x', name='get_file', type='function_call', id='fc_0487122c6a3abcdd006990e7d0c054819dbc895c734a8a3a03', status='completed'),
 ResponseFunctionToolCall(arguments='{"filename":"docs/platform/dashboard_overview.mdx"}', call_id='call_TN93Z8349kH2GQiV6ZCFBxSX', name='get_file', type='function_call', id='fc_0487122c6a3abcdd006990e7d0c05c819da99e8a35979da17e', status='completed')]

In [18]:
has_function_calls = False

for message in response.output:
    if message.type == 'function_call':
        has_function_calls = True
        print(f'executing {message.name}({message.arguments})...')
        tool_call_output = make_call(message)
        messages.append(tool_call_output)

    if message.type == 'message':
        text = message.content[0].text
        print('ASSISTANT:', text)

executing get_file({"filename":"docs/platform/dashboard_add_panels_ui.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_add_panels.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_overview.mdx"})...


In [19]:
messages.extend(response.output)

In [20]:
# 3rd call - final

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    tools=tools,
)

In [21]:
response.output

[ResponseOutputMessage(id='msg_0487122c6a3abcdd006990e80ca31c819d949d443c49f641df', content=[ResponseOutputText(annotations=[], text='To create a dashboard in Evidently, follow these steps:\n\n### 1. Initial Setup\n- Ensure you have run an evaluation and saved at least one Report to your Project. This provides the data source for your dashboard.\n\n### 2. Create a Dashboard\n- You can create dashboards through two main methods:\n  - **User Interface (UI)**: Available in Evidently Cloud and Enterprise, allowing you to add panels directly.\n  - **Python API**: If you prefer coding, you can define your dashboard as code.\n\n### 3. Adding Tabs\nYou can organize your dashboard using tabs. To add a new tab in the UI:\n- Enter "Edit" mode in the dashboard.\n- Click the plus sign to add a new tab and name it.\n\n### 4. Adding Panels\nPanels are visual elements in the dashboard. You can add various types of panels (text, counters, plots) to visualize data.\n\n#### In the UI:\n- Enter "Edit" mod

In [24]:
print(response.output_text)

To create a dashboard in Evidently, follow these steps:

### 1. Initial Setup
- Ensure you have run an evaluation and saved at least one Report to your Project. This provides the data source for your dashboard.

### 2. Create a Dashboard
- You can create dashboards through two main methods:
  - **User Interface (UI)**: Available in Evidently Cloud and Enterprise, allowing you to add panels directly.
  - **Python API**: If you prefer coding, you can define your dashboard as code.

### 3. Adding Tabs
You can organize your dashboard using tabs. To add a new tab in the UI:
- Enter "Edit" mode in the dashboard.
- Click the plus sign to add a new tab and name it.

### 4. Adding Panels
Panels are visual elements in the dashboard. You can add various types of panels (text, counters, plots) to visualize data.

#### In the UI:
- Enter "Edit" mode on your dashboard.
- Click "Add Panel".
- Configure your panel settings:
  - **Select Metrics**: Choose the metric from Reports that you want to visual

In [25]:
message_history = [
    {"role": "system", "content": common.SINGLE_TOOL_SYSTEM_PROMPT},
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

iteration_number = 1

while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=message_history,
        tools=tools,
    )

    print(f'iteration number {iteration_number}...')
    message_history.extend(response.output)

    has_function_calls = False

    for message in response.output:
        if message.type == 'function_call':
            print(f'executing {message.name}({message.arguments})...')
            tool_call_output = make_call(message)
            message_history.append(tool_call_output)
            has_function_calls = True

        if message.type == 'message':
            text = message.content[0].text
            print('ASSISTANT:', text)

    iteration_number = iteration_number + 1
    print()

    if not has_function_calls:
        break

iteration number 1...
executing search({"query":"create dashboard"})...

iteration number 2...
ASSISTANT: To create a dashboard in Evidently, you can utilize the following methods:

1. **No-Code Dashboards (UI)**:
   - You can design your dashboard by creating panels that visualize evaluation metrics without writing any code. For detailed instructions on how to add dashboard panels using the UI, refer to the [Add dashboard panels (UI)](docs/platform/dashboard_add_panels_ui.mdx) documentation.

2. **Code Approach (API)**:
   - If you prefer to manage dashboards programmatically, you can use APIs to create and manage dashboards. Check the [Add dashboard panels (API)](docs/platform/dashboard_add_panels.mdx) documentation for guidance.

3. **Panel Types**:
   - Familiarize yourself with the various panel types available for your dashboard. The [Dashboard panel types](docs/platform/dashboard_panel_types.mdx) documentation provides an overview of the different monitoring panels you can use.


In [26]:
from agent_responses import OpenAIResponsesAgent

In [27]:
agent = OpenAIResponsesAgent(
    llm_client=OpenAI(),
    model='gpt-4o-mini',
    instructions=common.SINGLE_TOOL_SYSTEM_PROMPT,
    tool_pairs=tool_pairs
)

In [28]:
messages = agent.loop(common.DEFAULT_QUESTION)

iteration number 1...
executing search({"query":"create dashboard"})...

iteration number 2...
executing get_file({"filename":"docs/platform/dashboard_add_panels_ui.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_add_panels.mdx"})...
executing get_file({"filename":"docs/platform/dashboard_overview.mdx"})...

iteration number 3...
ASSISTANT: To create a dashboard in Evidently, follow these steps:

### 1. **Initial Setup**
   - Ensure you have run an evaluation and saved at least one Report to your Project, as the Dashboard requires these Reports as a data source.

### 2. **Accessing the Dashboard**
   - Each Project has its own Dashboard, which is empty at first. You can visualise results through this Dashboard.

### 3. **Creating Dashboard Tabs**
   - You can organize Panels within your Dashboard into different Tabs. To add a Tab:
     - Enter "Edit" mode on the Dashboard (top right corner).
     - Click on the plus sign to add a new Tab.
     - You can either select 